<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Pytorch_WithDeeplizard/blob/master/23_CNN_Image_Prediction_With_PyTorch_Forward_Propagation_Explained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forward Propagation Explained - Using A PyTorch Neural Network

在这一节中，我们将看到如何使用卷积神经网络从数据集的样本图像生成输出预测张量。 事不宜迟，让我们开始吧。

在本系列的这一点上，我们已经完成了模型的构建，从技术上讲，我们可以从这里直接进入训练过程。 但是，让我们努力更好地了解我们的网络是如何开箱即用的，然后，一旦我们对网络有了更深入的了解，我们将更好地准备去了解训练过程。

第一步是了解正向传播。
## 1.什么是正向传播？

正向传播是将输入张量转换为输出张量的过程。 神经网络的核心是将输入张量映射到输出张量的功能，而正向传播只是将输入传递到网络并从网络接收输出的过程的特殊名称。

如我们所见，神经网络以张量的形式对数据进行操作。 前向传播的概念用于指示输入张量数据是通过网络在前向方向上传输的。

对于我们的网络，这意味着简单地将输入张量传递到网络并接收输出张量。 为此，我们将样本数据传递给网络的forward（）方法。

这就是为什么forward（）方法具有正向名称的原因，forward（）的执行是正向传播的过程。

如果您正在关注本系列，那么到现在为止我们知道，我们不会直接调用forward（）方法，而是会调用网络实例。 有关更多详细信息，查看前面几节。

前进这个词很直截了当。 ;）

但是，“传播”一词是指通过某种媒介移动或传播。 在神经网络的情况下，数据通过网络的各个层传播。

也有反向传播（backpropagation）的概念，这使得术语“正向传播”适合作为第一步。 在训练过程中，反向传播发生在正向传播之后。

在我们的情况下，从实际的角度来看，正向传播是将输入图像张量传递给我们在上一集中实现的forward（）方法的过程。 此输出是网络的预测。

在关于数据集和数据加载器的情节中，我们了解了如何从训练集中访问单个样本图像张量，更重要的是，如何从数据加载器访问一批图像张量。 现在我们已经定义了我们的网络并实现了forward（）方法：让图像传递到我们的网络以获得预测。


## 2.通过网络进行预测：正向传播
在此之前，我们将关闭PyTorch的梯度计算功能。这将阻止PyTorch在张量流经网络时自动构建计算图。

计算图通过跟踪每次发生的计算来跟踪网络的映射。 该图在训练过程中用于计算损失函数相对于网络权重的导数（梯度）。

由于我们尚未训练网络，因此我们不打算更新权重，因此不需要进行梯度计算。 在训练开始时，我们将重新打开此功能。

随着计算的发生，跟踪计算的过程实时发生。 记得在本系列开始时，我们曾说过PyTorch使用动态计算图。 现在，我们将其关闭。

严格来说，将其关闭不是必需的，但是由于该图形未存储在内存中，因此关闭该功能确实会减少内存消耗。 此代码将关闭该功能。


In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F


In [ ]:
torch.set_grad_enabled(False)

In [ ]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
    self.conv2=nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
    self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
    self.fc2 = nn.Linear(in_features=120,out_features=60)
    self.out = nn.Linear(in_features=60,out_features=10)
  
  def forward(self,t):
      t = t;

      t = self.conv1(t)
      t = F.relu(t)
      t = F.max_pool2d(t,kernel_size=2,stride=2)

      t = self.conv2(t)
      t = F.relu(t)
      t = F.max_pool2d(t,kernel_size=2,stride=2)
      
      t = t.reshape(-1,12*4*4)
      t = self.fc1(t)
      t = F.relu(t)

      t = self.fc2(t)
      t = F.relu(t)
      
      t = self.out(t)
      return t;
  

### 将单个图像传递到网络
---
让我们继续创建Network类的实例：

In [ ]:
network = Network()

接下来，我们将从训练集中获取一个样本，解压缩图像和标签，并验证图像的形状

In [ ]:
import torchvision
import torchvision.transforms as transforms


In [ ]:
train_set = torchvision.datasets.FashionMNIST(root='./data',train=True,
                                download=True,
                                transform=transforms.Compose([
        transforms.ToTensor()
    ]))

In [ ]:
train_loader =torch.utils.data.DataLoader(train_set,batch_size=1000,shuffle=True)

In [ ]:
sample = next(iter(train_set))
image,label = sample
image.shape

torch.Size([1, 28, 28])

图像张量的形状表明我们有一个高28，宽28的单通道图像。酷，这就是我们所期待的。

现在，我们必须执行第二步，然后才将张量传递给我们的网络。 当我们将张量传递到我们的网络时，网络正在等待一批，因此即使我们要传递单个图像，我们仍然需要一批。

没问题 我们可以创建一个包含单个图像的批处理。 所有这些都将打包成一个单一的四维张量，以反映以下尺寸。

* (batch_size, in_channels, height, width)

对网络的这种要求来自以下事实：nn.Conv2d卷积层类中的forward（）方法期望它们的进程具有4维。 这是相当标准的，因为大多数神经网络实现都处理成批的输入样本，而不是单个样本。

要将单个样本图像张量放入大小为1的批处理中，我们只需要unsqueeze（）张量即可添加一个额外的维数。 我们在前几集中看到了如何执行此操作。

In [ ]:
# Inserts an additional dimension that represents a batch of size 1
image.unsqueeze(0).shape

torch.Size([1, 1, 28, 28])

使用此功能，我们现在可以将unsqueezed的图像传递到我们的网络并获得网络的预测。

In [ ]:
pred = network(image.unsqueeze(0))
pred

tensor([[ 0.0091, -0.1273,  0.0050,  0.0021,  0.1367, -0.0328,  0.0402,  0.0194,
         -0.0389,  0.0916]])

In [ ]:
pred.shape

torch.Size([1, 10])

In [ ]:
label

9

In [ ]:
pred.argmax(dim=1)

tensor([4])

我们做到了！我们已经用我们的正向方法从网络中得到了一个预测。该网络已返回一个预测张量，其中包含十种服装中每种服装的预测值。

预测张量的形状为1×10。这告诉我们第一个轴的长度是1，而第二个轴的长度是10。对这一点的解释是，我们的批处理中有一个图像的十个预测类。也即是：
* （批量大小，预测类别数）

对于批次中的每个输入以及每个预测类别，我们都有一个预测值。 如果我们希望这些值是概率，则可以使用nn.functional软件包中的softmax（）函数。


In [ ]:
F.softmax(pred,dim=1)

tensor([[0.0996, 0.0869, 0.0992, 0.0989, 0.1132, 0.0955, 0.1028, 0.1007, 0.0950,
         0.1082]])

In [ ]:
F.softmax(pred,dim=1).sum()

tensor(1.)

训练集中第一张图像的标签为9，使用argmax（）函数，我们可以看到预测张量中的最大值出现在由索引7表示的类上。

预测：Coat（4）

当前：Ankle boot（9）

请记住，每个预测类都由一个对应的索引表示。

* 0	T-shirt/top
* 1	Trouser
* 2	Pullover
* 3	Dress
* 4	Coat
* 5	Sandal
* 6	Shirt
* 7	Sneaker
* 8	Bag
* 9	Ankle boot

这种情况下的预测是不正确的，这是我们所期望的，因为网络中的权重是随机生成的。

### 网络权重是随机生成的
---
关于这些结果，我们需要指出一些重要的事情。 大多数概率接近10％，这是有道理的，因为我们的网络正在猜测，并且我们有十个来自平衡数据集的预测类。

随机生成的权重的另一个含义是，每次我们创建网络的新实例时，网络内的权重都会不同。 这意味着，如果我们创建不同的网络，我们得到的预测将有所不同。 请记住这一点。 您的预测将与我们在此处看到的有所不同。

In [ ]:
net1 = Network()
net2 = Network()

In [ ]:
net1(image.unsqueeze(0))

tensor([[-0.0151, -0.0315,  0.0347, -0.0326, -0.1044,  0.0217,  0.0675, -0.0397,
         -0.0037,  0.1913]])

In [ ]:
net2(image.unsqueeze(0))

tensor([[-0.0318,  0.0588, -0.1327, -0.1346, -0.0400,  0.0624, -0.0164,  0.0792,
          0.0351,  0.0263]])

## 2.使用数据加载器传递批处理

现在，我们准备将一批数据传递到我们的网络并解释结果。

现在，我们应该对什么是正向传播以及如何将单个图像张量传递给PyTorch中的卷积神经网络有一个很好的了解。 在下一篇文章中，我们将看到如何使用数据加载器将批处理传递到我们的网络。 我会在那里见你！
